In [3]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

print("Libraries imported")

Libraries imported


## Part 1: Fetch the Postal Codes and Boroughs / Neighborhoods

In [4]:
boroughs_df = pd.read_excel("London_Boroughs.xlsx")
boroughs_df

,Code,Neighborhood
0,E09000001,City of London
1,E09000002,Barking and Dagenham
2,E09000003,Barnet
3,E09000004,Bexley
4,E09000005,Brent
5,E09000006,Bromley
6,E09000007,Camden
7,E09000008,Croydon
8,E09000009,Ealing
9,E09000010,Enfield


## Part 2: Fetch the latitude and longitude of the city boroughs

In [5]:
from geopy.geocoders import Nominatim

In [6]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
city_latitude = location.latitude
city_longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(city_latitude, city_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [7]:
latitudes = []
longitudes = []

for index, row in boroughs_df.iterrows():
    borough = row['Neighborhood']

    address = '{}, London, UK'.format(borough)

    location = geolocator.geocode(address)

    latitude = location.latitude
    longitude = location.longitude

    latitudes.append(latitude)
    longitudes.append(longitude)

boroughs_df['Latitude'] = latitudes
boroughs_df['Longitude'] = longitudes


In [8]:
boroughs_df

,Code,Neighborhood,Latitude,Longitude
0,E09000001,City of London,51.515618,-0.091998
1,E09000002,Barking and Dagenham,51.554117,0.150504
2,E09000003,Barnet,51.653090,-0.200226
3,E09000004,Bexley,51.441679,0.150488
4,E09000005,Brent,51.563826,-0.275760
5,E09000006,Bromley,51.402805,0.014814
6,E09000007,Camden,51.542305,-0.139560
7,E09000008,Croydon,51.371305,-0.101957
8,E09000009,Ealing,51.512655,-0.305195
9,E09000010,Enfield,51.652085,-0.081018


## Part 3: Explore and Cluster city boroughs/neighborhoods

## 3.1. Visualize city neighborhoods

In [9]:
address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
city_latitude = location.latitude
city_longitude = location.longitude

In [10]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
# create map of New York using latitude and longitude values
map_city = folium.Map(location=[city_latitude, city_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], boroughs_df['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city

## 3.2 Explore City Neighborhoods

In [12]:
CLIENT_ID = 'G3I1S3STTCK0OMQXUGZPMWJZ2LAZN0AUYT0NB23OBVG1TJEU' # your Foursquare ID
CLIENT_SECRET = 'NOOBRTQ22EA3OPFVBE5ULOXSXX3MTMHKDQLWLQMT3M0ACUXR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G3I1S3STTCK0OMQXUGZPMWJZ2LAZN0AUYT0NB23OBVG1TJEU
CLIENT_SECRET:NOOBRTQ22EA3OPFVBE5ULOXSXX3MTMHKDQLWLQMT3M0ACUXR


In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to explore all the neighborhoods in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        print('{}: {} venues'.format(name, len(results)))

        
        # return only relevant information for each nearby venue

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
neighborhood_names = boroughs_df.loc[:, 'Neighborhood'] # neighborhood names
neighborhood_latitudes = boroughs_df.loc[:, 'Latitude'] # neighborhood latitude values
neighborhood_longitudes = boroughs_df.loc[:, 'Longitude'] # neighborhood longitude values

boroughs_venues = getNearbyVenues(neighborhood_names, neighborhood_latitudes, neighborhood_longitudes)

boroughs_venues.head()

City of London: 100 venues
Barking and Dagenham: 18 venues
Barnet: 30 venues
Bexley: 10 venues
Brent: 73 venues
Bromley: 58 venues
Camden: 100 venues
Croydon: 93 venues
Ealing: 100 venues
Enfield: 44 venues
Greenwich: 100 venues
Hackney: 100 venues
Hammersmith and Fulham: 100 venues
Haringey: 42 venues
Harrow: 16 venues
Havering: 100 venues
Hillingdon: 11 venues
Hounslow: 65 venues
Islington: 100 venues
Kensington and Chelsea: 100 venues
Kingston upon Thames: 100 venues
Lambeth: 100 venues
Lewisham: 35 venues
Merton: 59 venues
Newham: 30 venues
Redbridge: 10 venues
Richmond upon Thames: 25 venues
Southwark: 100 venues
Sutton: 6 venues
Tower Hamlets: 36 venues
Waltham Forest: 24 venues
Wandsworth: 70 venues
Westminster: 100 venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.515618,-0.091998,The Ned Hotel,51.513755,-0.090067,Hotel
1,City of London,51.515618,-0.091998,Hawksmoor Guildhall,51.515498,-0.090849,Steakhouse
2,City of London,51.515618,-0.091998,Goodman Steak House Restaurant,51.514398,-0.090745,Steakhouse
3,City of London,51.515618,-0.091998,The Merchant House,51.513264,-0.093039,Cocktail Bar
4,City of London,51.515618,-0.091998,Daunt Books,51.513982,-0.092995,Bookstore


## 3.3 Analyze neighborhoods Venues

In [16]:
print(boroughs_venues.shape)
print(len(boroughs_venues['Neighborhood'].unique()))
boroughs_venues.head()

(2055, 7)
33


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.515618,-0.091998,The Ned Hotel,51.513755,-0.090067,Hotel
1,City of London,51.515618,-0.091998,Hawksmoor Guildhall,51.515498,-0.090849,Steakhouse
2,City of London,51.515618,-0.091998,Goodman Steak House Restaurant,51.514398,-0.090745,Steakhouse
3,City of London,51.515618,-0.091998,The Merchant House,51.513264,-0.093039,Cocktail Bar
4,City of London,51.515618,-0.091998,Daunt Books,51.513982,-0.092995,Bookstore


In [17]:
boroughs_venues.shape

(2055, 7)

In [18]:
boroughs_venues.groupby('Neighborhood').count() # how many venues were retruned for each borough

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham,18,18,18,18,18,18
Barnet,30,30,30,30,30,30
Bexley,10,10,10,10,10,10
Brent,73,73,73,73,73,73
Bromley,58,58,58,58,58,58
Camden,100,100,100,100,100,100
City of London,100,100,100,100,100,100
Croydon,93,93,93,93,93,93
Ealing,100,100,100,100,100,100


In [19]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(boroughs_venues['Venue Category'].unique())))

There are 252 uniques categories.


In [20]:
boroughs_venues[boroughs_venues['Venue Category'] == "Neighborhood"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [21]:
# drop rows containing Venue Category = "Neighborhood", it has no meaning!
boroughs_venues = boroughs_venues[boroughs_venues['Venue Category'] != "Neighborhood"]

#### Now create one hot encoded data frame with the categories for each neighborhood

In [22]:
# one hot encoding
boroughs_onehot = pd.get_dummies(boroughs_venues[['Venue Category']], prefix="", prefix_sep="")

boroughs_onehot.head()

,Afghan Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
# Re-add Neighborhood column and make it the first column
columns_names = list(boroughs_onehot.columns.values)

boroughs_onehot['Neighborhood'] = boroughs_venues['Neighborhood']

columns_names.insert(0, 'Neighborhood')

boroughs_onehot = boroughs_onehot[columns_names]

boroughs_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
boroughs_onehot[boroughs_onehot['Neighborhood'] == 'Barking and Dagenham']

,Neighborhood,Afghan Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
100,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
boroughs_onehot.shape

(2055, 253)

In [26]:
boroughs_grouped = boroughs_onehot.groupby('Neighborhood').mean().reset_index()
boroughs_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Observatory,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barking and Dagenham,0.0000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0000

In [27]:
boroughs_grouped.shape

(33, 253)

In [28]:
def return_most_common_venues(row, c):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    #print(row_categories_sorted)

    sorted_categories_ar = row_categories_sorted.index.values[0:c]

    for i, value in enumerate(row_categories_sorted):
        if value == 0 and i < c:
            sorted_categories_ar[i] = ''
        elif i == c:
            break
    
    return sorted_categories_ar

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boroughs_grouped['Neighborhood']

for ind in np.arange(boroughs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boroughs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Gas Station,Grocery Store,Bus Stop,Pub,Turkish Restaurant,Supermarket,Park,Soccer Stadium,Chinese Restaurant,Golf Course
1,Barnet,Coffee Shop,Pub,Grocery Store,Pizza Place,Bookstore,Pharmacy,Fast Food Restaurant,Bar,Park,Market
2,Bexley,Pub,Italian Restaurant,Hotel,Steakhouse,Tennis Court,Fast Food Restaurant,Greek Restaurant,Toy / Game Store,Indian Restaurant,
3,Brent,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Hotel,Sandwich Place,Supermarket,Food Court,Warehouse Store,Indian Restaurant
4,Bromley,Clothing Store,Pub,Coffee Shop,Gym / Fitness Center,Burger Joint,Park,Pizza Place,Bar,Supermarket,Indian Restaurant


In [30]:
neighborhoods_venues_sorted.shape

(33, 11)

## 3.4. Cluster Neighborhoods

In [31]:
# set number of clusters
kclusters = 4

boroughs_grouped_clustering = boroughs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 0,
       2, 2, 1, 2, 2, 0, 3, 2, 2, 2, 0], dtype=int32)

In [32]:
kmeans.labels_.shape

(33,)

In [33]:
# add clustering labels
try:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop("Cluster Labels", 1)
except:
    print("Cluster Labels not present")
    
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

Cluster Labels not present


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Barking and Dagenham,Gas Station,Grocery Store,Bus Stop,Pub,Turkish Restaurant,Supermarket,Park,Soccer Stadium,Chinese Restaurant,Golf Course
1,2,Barnet,Coffee Shop,Pub,Grocery Store,Pizza Place,Bookstore,Pharmacy,Fast Food Restaurant,Bar,Park,Market
2,2,Bexley,Pub,Italian Restaurant,Hotel,Steakhouse,Tennis Court,Fast Food Restaurant,Greek Restaurant,Toy / Game Store,Indian Restaurant,
3,0,Brent,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Hotel,Sandwich Place,Supermarket,Food Court,Warehouse Store,Indian Restaurant
4,2,Bromley,Clothing Store,Pub,Coffee Shop,Gym / Fitness Center,Burger Joint,Park,Pizza Place,Bar,Supermarket,Indian Restaurant
5,2,Camden,Pub,Coffee Shop,Café,Grocery Store,Music Venue,Supermarket,Italian Restaurant,Market,Falafel Restaurant,Flea Market
6,0,City of London,Gym / Fitness Center,Coffee Shop,Hotel,Cocktail Bar,Modern European Restaurant,Steakhouse,Italian Restaurant,Indie Movie Theater,Theater,Scenic Lookout
7,2,Croydon,Coffee Shop,Pub,Platform,Clothing Store,Italian Restaurant,Café,Mediterranean Restaurant,Hotel,Bus Stop,Bookstore
8,2,Ealing,Coffee Shop,Pub,Café,Italian Restaurant,Park,Thai Restaurant,Pizza Place,Bakery,Hotel,Burger Joint
9,2,Enfield,Pub,Coffee Shop,Clothing Store,Department Store,Fish & Chips Shop,Indian Restaurant,Pharmacy,Park,Supermarket,Bookstore


In [34]:
boroughs_merged = boroughs_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
boroughs_merged = neighborhoods_venues_sorted.join(boroughs_merged.set_index('Neighborhood'), on='Neighborhood')

boroughs_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
0,1,Barking and Dagenham,Gas Station,Grocery Store,Bus Stop,Pub,Turkish Restaurant,Supermarket,Park,Soccer Stadium,Chinese Restaurant,Golf Course,E09000002,51.554117,0.150504
1,2,Barnet,Coffee Shop,Pub,Grocery Store,Pizza Place,Bookstore,Pharmacy,Fast Food Restaurant,Bar,Park,Market,E09000003,51.653090,-0.200226
2,2,Bexley,Pub,Italian Restaurant,Hotel,Steakhouse,Tennis Court,Fast Food Restaurant,Greek Restaurant,Toy / Game Store,Indian Restaurant,,E09000004,51.441679,0.150488
3,0,Brent,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Hotel,Sandwich Place,Supermarket,Food Court,Warehouse Store,Indian Restaurant,E09000005,51.563826,-0.275760
4,2,Bromley,Clothing Store,Pub,Coffee Shop,Gym / Fitness Center,Burger Joint,Park,Pizza Place,Bar,Supermarket,Indian Restaurant,E09000006,51.402805,0.014814


#### let's visualize the resulting clusters

In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boroughs_merged['Latitude'], boroughs_merged['Longitude'], boroughs_merged['Neighborhood'], boroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(label)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.5. Examine Clusters

In [36]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 0, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
3,Brent,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Hotel,Sandwich Place,Supermarket,Food Court,Warehouse Store,Indian Restaurant,E09000005,51.563826,-0.275760
6,City of London,Gym / Fitness Center,Coffee Shop,Hotel,Cocktail Bar,Modern European Restaurant,Steakhouse,Italian Restaurant,Indie Movie Theater,Theater,Scenic Lookout,E09000001,51.515618,-0.091998
21,Lambeth,Hotel,Theater,Coffee Shop,Garden,Café,Event Space,Plaza,Pub,Scenic Lookout,Bar,E09000022,51.501301,-0.117287
27,Southwark,Hotel,Coffee Shop,Theater,Pub,Bakery,Seafood Restaurant,Italian Restaurant,Street Food Gathering,Scenic Lookout,Art Museum,E09000028,51.502922,-0.103458
32,Westminster,Hotel,Café,Coffee Shop,Plaza,Garden,Monument / Landmark,Outdoor Sculpture,Hotel Bar,Park,Historic Site,E09000033,51.500444,-0.126540


In [37]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 1, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
0,Barking and Dagenham,Gas Station,Grocery Store,Bus Stop,Pub,Turkish Restaurant,Supermarket,Park,Soccer Stadium,Chinese Restaurant,Golf Course,E09000002,51.554117,0.150504
14,Harrow,Indian Restaurant,Grocery Store,Supermarket,Coffee Shop,Bus Stop,Park,Thai Restaurant,Fast Food Restaurant,Gym,Gym / Fitness Center,E09000015,51.596827,-0.337316
16,Hillingdon,Chinese Restaurant,Fast Food Restaurant,Construction & Landscaping,Plaza,Coffee Shop,Park,Pharmacy,Grocery Store,Pub,,E09000017,51.542519,-0.448335
17,Hounslow,Indian Restaurant,Grocery Store,Coffee Shop,Hotel,Fast Food Restaurant,Clothing Store,Discount Store,Bakery,Sandwich Place,Supermarket,E09000018,51.468613,-0.361347
24,Newham,Bus Stop,Grocery Store,Pub,Fish & Chips Shop,Gym / Fitness Center,Toy / Game Store,Convenience Store,Sandwich Place,Coffee Shop,Pharmacy,E09000025,51.530000,0.029318


In [38]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 2, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
1,Barnet,Coffee Shop,Pub,Grocery Store,Pizza Place,Bookstore,Pharmacy,Fast Food Restaurant,Bar,Park,Market,E09000003,51.653090,-0.200226
2,Bexley,Pub,Italian Restaurant,Hotel,Steakhouse,Tennis Court,Fast Food Restaurant,Greek Restaurant,Toy / Game Store,Indian Restaurant,,E09000004,51.441679,0.150488
4,Bromley,Clothing Store,Pub,Coffee Shop,Gym / Fitness Center,Burger Joint,Park,Pizza Place,Bar,Supermarket,Indian Restaurant,E09000006,51.402805,0.014814
5,Camden,Pub,Coffee Shop,Café,Grocery Store,Music Venue,Supermarket,Italian Restaurant,Market,Falafel Restaurant,Flea Market,E09000007,51.542305,-0.139560
7,Croydon,Coffee Shop,Pub,Platform,Clothing Store,Italian Restaurant,Café,Mediterranean Restaurant,Hotel,Bus Stop,Bookstore,E09000008,51.371305,-0.101957
8,Ealing,Coffee Shop,Pub,Café,Italian Restaurant,Park,Thai Restaurant,Pizza Place,Bakery,Hotel,Burger Joint,E09000009,51.512655,-0.305195
9,Enfield,Pub,Coffee Shop,Clothing Store,Department Store,Fish & Chips Shop,Indian Restaurant,Pharmacy,Park,Supermarket,Bookstore,E09000010,51.652085,-0.081018
10,Greenwich,Pub,Grocery Store,Garden,Café,Gym / Fitness Center,Park,Pizza Place,Indian Restaurant,Bar,Bakery,E09000011,51.482084,-0.004542
11,Hackney,Pub,Bakery,Café,Brewery,Coffee Shop,Park,Cocktail Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Gym / Fitness Center,E09000012,51.543240,-0.049362
12,Hammersmith and Fulham,Pub,Coffee Shop,Gym / Fitness Center,Café,Sandwich Place,Italian Restaurant,Thai Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,E09000013,51.492038,-0.223640


In [39]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 3, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
28,Sutton,Grocery Store,Italian Restaurant,Supermarket,Pub,Park,,,,,,E09000029,51.357464,-0.173627


In [40]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 4, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [41]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 5, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [42]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 6, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [43]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 7, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [44]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 8, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [45]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 9, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [46]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 10, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude


In [47]:
boroughs_merged.loc[boroughs_merged['Cluster Labels'] == 11, boroughs_merged.columns[[1] + list(range(2, boroughs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Latitude,Longitude
